In [180]:
import pandas as pd
import numpy as np
import math

In [181]:
data = pd.read_csv('id3.csv')
data.head()

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes


In [182]:
train_set = data.head(int(len(data)*0.7))
train_set

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes


In [183]:
test_set = data.loc[data.index > len(train_set)-1]
test_set

,Outlook,Temperature,Humidity,Wind,Answer
9,rain,mild,normal,weak,yes
10,sunny,mild,normal,strong,yes
11,overcast,mild,high,strong,yes
12,overcast,hot,normal,weak,yes
13,rain,mild,high,strong,no


In [184]:
p1 = len(data.where(data[data.columns.tolist()[-1]] == 'yes').dropna())
n1 = len(data.where(data[data.columns.tolist()[-1]] == 'no').dropna())
p=p1/(p1+n1)
n=n1/(p1+n1)
print(p,n)

0.6428571428571429 0.35714285714285715


In [185]:
attributes = data.columns.tolist()
attributes = attributes[0:-1]
attributes

['Outlook', 'Temperature', 'Humidity', 'Wind']

In [186]:
values = {}
for i in attributes:
    values[i] = data[i].unique()

In [187]:
from collections import defaultdict
#function to create a multidimensional dictionary
def multi_dict(K, type):
    if K == 1:
        return defaultdict(type)
    else:
        return defaultdict(lambda: multi_dict(K-1, type))

In [192]:
curr_prob = {}
curr_prob = multi_dict(3, float)
for i in attributes:
    sum=0
    for j in values[i]:
        curr_prob_yes=len(data.where((data[i]==j) & (data[data.columns.tolist()[-1]]=='yes')).dropna())
        curr_prob_no=len(data.where((data[i]==j) & (data[data.columns.tolist()[-1]]=='no')).dropna())
        curr_prob[i][j]['yes']=curr_prob_yes/p1
        curr_prob[i][j]['no']=curr_prob_no/n1

In [193]:
def test_model(test_data):
    unsuccessfull_count = 0
    for index,i in test_data.iterrows():
        v_yes,v_no,res = p,n,-1
        x=0
        for j in attributes:
            v_yes *= curr_prob[j][i[x]]['yes']
            v_no *= curr_prob[j][i[x]]['no']
            x+=1
        v_yes = v_yes/(v_yes + v_no)
        v_no = v_no/(v_yes + v_no)
        if(v_yes >= v_no):
            res='yes'
        else:
            res='no'
        if(res != i[-1]):
            unsuccessfull_count += 1
    print("Accuracy of Naive Bayes classifier : ",((len(test_data)-unsuccessfull_count)/len(test_data))*100,"%")
test_model(test_set)

Accuracy of Naive Bayes classifier :  80.0 %
